In [1]:
import polars as pl
import numpy as np

from sklearn import preprocessing
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.model_selection import train_test_split

In [19]:
%%time
innweeks = pl.read_parquet("data/target_series.parquet"
                            ).with_columns(                              
                                pl.col("week").cast(pl.Int8),
                                np.log1p(pl.col("target")),
                            )
innweeks

CPU times: user 222 ms, sys: 107 ms, total: 328 ms
Wall time: 149 ms


inn_id,week,target
str,i8,f64
"""inn1000051""",0,12.95314
"""inn1000051""",1,7.392578
"""inn1000051""",2,13.929488
"""inn1000051""",3,10.583951
"""inn1000051""",4,12.972155
…,…,…
"""inn999886""",101,15.637834
"""inn999886""",102,17.042798
"""inn999886""",103,16.736929


In [6]:
#innweeks.group_by("inn_id").agg( pl.col("week").count()-pl.col("week").max()+pl.col("week").min() )["week"].value_counts()

In [28]:
%%time
innweeks = innweeks.with_columns(
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(1).over("inn_id").alias("imax1"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(2).over("inn_id").alias("imax2"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(3).over("inn_id").alias("imax3"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(4).over("inn_id").alias("imax4"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(5).over("inn_id").alias("imax5"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(6).over("inn_id").alias("imax6"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(7).over("inn_id").alias("imax7"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(8).over("inn_id").alias("imax8"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(9).over("inn_id").alias("imax9"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(10).over("inn_id").alias("imax10"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(11).over("inn_id").alias("imax11"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(12).over("inn_id").alias("imax12"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(13).over("inn_id").alias("imax13"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(14).over("inn_id").alias("imax14"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(15).over("inn_id").alias("imax15"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(16).over("inn_id").alias("imax16"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(17).over("inn_id").alias("imax17"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(18).over("inn_id").alias("imax18"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(19).over("inn_id").alias("imax19"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(20).over("inn_id").alias("imax20"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(21).over("inn_id").alias("imax21"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(22).over("inn_id").alias("imax22"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(23).over("inn_id").alias("imax23"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(24).over("inn_id").alias("imax24"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(25).over("inn_id").alias("imax25"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(26).over("inn_id").alias("imax26"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(27).over("inn_id").alias("imax27"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(28).over("inn_id").alias("imax28"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(29).over("inn_id").alias("imax29"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(30).over("inn_id").alias("imax30"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(31).over("inn_id").alias("imax31"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(32).over("inn_id").alias("imax32"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(33).over("inn_id").alias("imax33"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(34).over("inn_id").alias("imax34"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(35).over("inn_id").alias("imax35"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(36).over("inn_id").alias("imax36"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(37).over("inn_id").alias("imax37"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(38).over("inn_id").alias("imax38"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(39).over("inn_id").alias("imax39"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(40).over("inn_id").alias("imax40"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(41).over("inn_id").alias("imax41"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(42).over("inn_id").alias("imax42"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(43).over("inn_id").alias("imax43"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(44).over("inn_id").alias("imax44"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(45).over("inn_id").alias("imax45"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(46).over("inn_id").alias("imax46"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(47).over("inn_id").alias("imax47"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(48).over("inn_id").alias("imax48"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(49).over("inn_id").alias("imax49"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(50).over("inn_id").alias("imax50"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(51).over("inn_id").alias("imax51"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(52).over("inn_id").alias("imax52"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(53).over("inn_id").alias("imax53"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(54).over("inn_id").alias("imax54"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(55).over("inn_id").alias("imax55"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(56).over("inn_id").alias("imax56"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(57).over("inn_id").alias("imax57"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(58).over("inn_id").alias("imax58"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(59).over("inn_id").alias("imax59"),
                      
                      pl.col("target").rolling_max(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imax"),
                      pl.col("target").rolling_min(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imin"),
                      pl.col("target").rolling_mean(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imean"),
                      pl.col("target").rolling_median(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imedian"),
    
                      pl.col("target").rolling_quantile(quantile=0.95, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq095"),
                      pl.col("target").rolling_quantile(quantile=0.90, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq090"),
                      pl.col("target").rolling_quantile(quantile=0.85, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq085"),
                      pl.col("target").rolling_quantile(quantile=0.80, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq080"),
                      pl.col("target").rolling_quantile(quantile=0.75, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq075"),
                      pl.col("target").rolling_quantile(quantile=0.70, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq070"),
                      pl.col("target").rolling_quantile(quantile=0.65, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq065"),
                      pl.col("target").rolling_quantile(quantile=0.60, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq060"),
                      pl.col("target").rolling_quantile(quantile=0.55, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq055"),
    
                      pl.col("target").rolling_quantile(quantile=0.45, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq045"),
                      pl.col("target").rolling_quantile(quantile=0.40, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq040"),
                      pl.col("target").rolling_quantile(quantile=0.35, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq035"),
                      pl.col("target").rolling_quantile(quantile=0.30, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq030"),
                      pl.col("target").rolling_quantile(quantile=0.25, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq025"),
                      pl.col("target").rolling_quantile(quantile=0.20, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq020"),
                      pl.col("target").rolling_quantile(quantile=0.15, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq015"),
                      pl.col("target").rolling_quantile(quantile=0.10, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq010"),
                      pl.col("target").rolling_quantile(quantile=0.05, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq005"),
                     )
innweeks

CPU times: user 16.6 s, sys: 6.25 s, total: 22.9 s
Wall time: 22.5 s


inn_id,week,target,imax,imin,imean,imedian,iq095,iq090,iq085,iq080,iq075,iq070,iq065,iq060,iq055,iq045,iq040,iq035,iq030,iq025,iq020,iq015,iq010,iq005,imax1,imax2,imax3,imax4,imax5,imax6,imax7,imax8,imax9,imax10,imax11,imax12,imax13,imax14,imax15,imax16
str,i8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""inn1000051""",0,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""inn1000051""",1,7.392578,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""inn1000051""",2,13.929488,12.95314,7.392578,10.172859,10.172859,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""inn1000051""",3,10.583951,13.929488,7.392578,11.425069,12.95314,13.929488,13.929488,13.929488,13.929488,13.929488,13.929488,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,13.929488,7.392578,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null
"""inn1000051""",4,12.972155,13.929488,7.392578,11.214789,11.768546,13.929488,13.929488,13.929488,13.929488,13.929488,12.95314,12.95314,12.95314,12.95314,10.583951,10.583951,10.583951,10.583951,10.583951,7.392578,7.392578,7.392578,7.392578,10.583951,13.929488,7.392578,12.95314,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn999886""",101,15.637834,17.231515,11.500407,16.103303,16.193706,16.902591,16.789835,16.786354,16.653305,16.540138,16.509635,16.414182,16.347232,16.26866,16.120146,16.104969,16.08884,15.998248,15.974485,15.932649,15.602697,15.362628,14.700636,16.703016,16.853691,16.48522,16.193706,16.48499,16.786354,16.414182,15.974485,16.120146,16.788546,16.801483,15.919157,15.546629,15.973451,16.08884,16.104969
"""inn999886""",102,17.042798,17.231515,11.500407,16.094803,16.193706,16.902591,16.789835,16.786354,16.653305,16.540138,16.509635,16.414182,16.347232,16.26866,16.120146,16.104969,16.08884,15.992497,15.973451,15.919157,15.602697,15.362628,14.700636,15.637834,16.703016,16.853691,16.48522,16.193706,16.48499,16.786354,16.414182,15.974485,16.120146,16.788546,16.801483,15.919157,15.546629,15.973451,16.08884
"""inn999886""",103,16.736929,17.231515,11.500407,16.104288,16.193706,16.993851,16.801483,16.788546,16.703016,16.586483,16.509635,16.414182,16.347232,16.26866,16.120146,16.104969,16.08884,15.992497,15.973451,15.919157,15.602697,15.362628,14.700636,17.042798,15.637834,16.703016,16.853691,16.48522,16.193706,16.48499,16.786354,16.414182,15.974485,16.120146,16.788546,16.801483,15.919157,15.546629,15.973451


In [8]:
calendar = pl.read_csv("data/calendar.csv", try_parse_dates=True
                      ).with_columns(
                              pl.col("week").cast(pl.Int8),
)
calendar

date,week,part
date,i8,str
2022-07-25,0,"""train"""
2022-07-26,0,"""train"""
2022-07-27,0,"""train"""
2022-07-28,0,"""train"""
2022-07-29,0,"""train"""
…,…,…
2024-10-23,117,"""private"""
2024-10-24,117,"""private"""
2024-10-25,117,"""private"""


In [9]:
weeks = calendar.group_by("week").agg(
                              pl.col("date").min().alias("d1"),
                              pl.col("date").max().alias("d2"),
                             ).with_columns(
                              pl.col("d1").dt.year().cast(pl.Int16).alias("year1"),
                              pl.col("d1").dt.month().alias("month1"),
                              pl.col("d1").dt.day().alias("day1"),
                              pl.col("d1").dt.ordinal_day().alias("dy1"),
    
                              pl.col("d2").dt.year().cast(pl.Int16).alias("year2"),
                              pl.col("d2").dt.month().alias("month2"),
                              pl.col("d2").dt.day().alias("day2"),
                              pl.col("d2").dt.ordinal_day().alias("dy2"),
                             ).with_columns(
                              ((pl.col("month2")-pl.col("month1")+12)%12).alias("m"),
                              (pl.col("year2")-pl.col("year1")).cast(pl.Int8).alias("y"),
    
                              pl.when( (pl.col("month1")>2)&(pl.col("year1")%4==0)).then(pl.col("dy1")-1).otherwise(pl.col("dy1")).alias("dy1"),
                              pl.when( (pl.col("month2")>2)&(pl.col("year2")%4==0)).then(pl.col("dy2")-1).otherwise(pl.col("dy2")).alias("dy2"),
                             )
weeks

week,d1,d2,year1,month1,day1,dy1,year2,month2,day2,dy2,m,y
i8,date,date,i16,i8,i8,i16,i16,i8,i8,i16,i8,i8
57,2023-08-28,2023-09-03,2023,8,28,240,2023,9,3,246,1,0
4,2022-08-22,2022-08-28,2022,8,22,234,2022,8,28,240,0,0
40,2023-05-01,2023-05-07,2023,5,1,121,2023,5,7,127,0,0
30,2023-02-20,2023-02-26,2023,2,20,51,2023,2,26,57,0,0
0,2022-07-25,2022-07-31,2022,7,25,206,2022,7,31,212,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…
62,2023-10-02,2023-10-08,2023,10,2,275,2023,10,8,281,0,0
113,2024-09-23,2024-09-29,2024,9,23,266,2024,9,29,272,0,0
39,2023-04-24,2023-04-30,2023,4,24,114,2023,4,30,120,0,0


In [11]:
profiles = pl.read_parquet("data/profiles.parquet")#["diff_datopen_report_date_flg"].unique()
age={
    "1m":1,
    "2_3m":3,
    "3_6m":6,
    "6_12m":12,
    "1_2y":24,
    "2_4y":48,
    "4_8y":96,
    "8_12y":144,
    "more_12y":192,
    #"NULL":np.nan
}    
ipul={
    "ip":0,
    "ul":1,
}    
profiles = profiles.with_columns(
    pl.col("diff_datopen_report_date_flg").replace(age, default=np.NAN),
    pl.col("ipul").replace(ipul).cast(pl.String),
    pl.col("id_region").fill_null("-"),
    pl.col("main_okved_group").fill_null("-"),
)
profiles = profiles.sort("report_date").group_by("inn_id").last()
profiles

/tmp/ipykernel_54905/699668050.py:19: DeprecationWarning: The `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
  pl.col("diff_datopen_report_date_flg").replace(age, default=np.NAN),


inn_id,report_date,ipul,id_region,main_okved_group,diff_datopen_report_date_flg
str,str,str,str,str,f64
"""inn3263536""","""2024-08-01""","""1""","""14""","""49""",144.0
"""inn2248091""","""2024-08-01""","""0""","""86""","""49""",192.0
"""inn2823786""","""2024-08-01""","""0""","""4""","""68""",48.0
"""inn548785""","""2024-08-01""","""1""","""22""","""36""",144.0
"""inn841482""","""2024-08-01""","""0""","""71""","""25""",48.0
…,…,…,…,…,…
"""inn1716660""","""2024-08-01""","""1""","""75""","""18""",144.0
"""inn5190698""","""2024-08-01""","""1""","""46""","""45""",144.0
"""inn2716569""","""2024-08-01""","""1""","""45""","""71""",24.0


In [29]:
XX = innweeks.join(weeks,
                   on="week",
                   how="left"
                  ).join(profiles,on="inn_id",
                         how="left"
                        )#.filter(pl.col("week")>60)
XX

inn_id,week,target,imax,imin,imean,imedian,iq095,iq090,iq085,iq080,iq075,iq070,iq065,iq060,iq055,iq045,iq040,iq035,iq030,iq025,iq020,iq015,iq010,iq005,imax1,imax2,imax3,imax4,imax5,imax6,imax7,imax8,imax9,imax10,imax11,imax12,imax13,imax14,imax15,imax16,d1,d2,year1,month1,day1,dy1,year2,month2,day2,dy2,m,y,report_date,ipul,id_region,main_okved_group,diff_datopen_report_date_flg
str,i8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,date,date,i16,i8,i8,i16,i16,i8,i8,i16,i8,i8,str,str,str,str,f64
"""inn1000051""",0,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,206,2022,7,31,212,0,0,"""2024-08-01""","""0""","""45""","""68""",96.0
"""inn1000051""",1,7.392578,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-08-01,2022-08-07,2022,8,1,213,2022,8,7,219,0,0,"""2024-08-01""","""0""","""45""","""68""",96.0
"""inn1000051""",2,13.929488,12.95314,7.392578,10.172859,10.172859,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-08-08,2022-08-14,2022,8,8,220,2022,8,14,226,0,0,"""2024-08-01""","""0""","""45""","""68""",96.0
"""inn1000051""",3,10.583951,13.929488,7.392578,11.425069,12.95314,13.929488,13.929488,13.929488,13.929488,13.929488,13.929488,12.95314,12.95314,12.95314,12.95314,12.95314,12.95314,7.392578,7.392578,7.392578,7.392578,7.392578,7.392578,13.929488,7.392578,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,2022-08-15,2022-08-21,2022,8,15,227,2022,8,21,233,0,0,"""2024-08-01""","""0""","""45""","""68""",96.0
"""inn1000051""",4,12.972155,13.929488,7.392578,11.214789,11.768546,13.929488,13.929488,13.929488,13.929488,13.929488,12.95314,12.95314,12.95314,12.95314,10.583951,10.583951,10.583951,10.583951,10.583951,7.392578,7.392578,7.392578,7.392578,10.583951,13.929488,7.392578,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,2022-08-22,2022-08-28,2022,8,22,234,2022,8,28,240,0,0,"""2024-08-01""","""0""","""45""","""68""",96.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn999886""",101,15.637834,17.231515,11.500407,16.103303,16.193706,16.902591,16.789835,16.786354,16.653305,16.540138,16.509635,16.414182,16.347232,16.26866,16.120146,16.104969,16.08884,15.998248,15.974485,15.932649,15.602697,15.362628,14.700636,16.703016,16.853691,16.48522,16.193706,16.48499,16.786354,16.414182,15.974485,16.120146,16.788546,16.801483,15.919157,15.546629,15.973451,16.08884,16.104969,2024-07-01,2024-07-07,2024,7,1,182,2024,7,7,188,0,0,"""2024-08-01""","""1""","""22""","""45""",192.0
"""inn999886""",102,17.042798,17.231515,11.500407,16.094803,16.193706,16.902591,16.789835,16.786354,16.653305,16.540138,16.509635,16.414182,16.347232,16.26866,16.120146,16.104969,16.08884,15.992497,15.973451,15.919157,15.602697,15.362628,14.700636,15.637834,16.703016,16.853691,16.48522,16.193706,16.48499,16.786354,16.414182,15.974485,16.120146,16.788546,16.801483,15.919157,15.546629,15.973451,16.08884,2024-07-08,2024-07-14,2024,7,8,189,2024,7,14,195,0,0,"""2024-08-01""","""1""","""22""","""45""",192.0
"""inn999886""",103,16.736929,17.231515,11.500407,16.104288,16.193706,16.993851,16.801483,16.788546,16.703016,16.586483,16.509635,16.414182,16.347232,16.26866,16.120146,16.104969,16.08884,15.992497,15.973451,15.919157,15.602697,15.362628,14.7006

In [30]:
y = XX["target"].to_numpy()
X = XX.drop(["target",
             "d1",
             "d2",
             #"week",
             #"inn_id",
             "report_date"
            ]).to_pandas()
#X_train, X_test, y_train, y_test = train_test_split(    X, y, test_size=0.01, random_state=42)
border=102
X_train = X[X["week"]<=border].drop("week",axis=1) 
X_test = X[X["week"]>border].drop("week",axis=1) 
y_train = y[X["week"]<=border]  
y_test = y[X["week"]>border] 

In [39]:
model = CatBoostRegressor(
                            #custom_metric="MSLE",
                            #eval_metric="MSLE",
                            #iterations=3000,
                            #learning_rate=0.20126033,
                            depth=6,
                            cat_features=["inn_id",
                                          "ipul",
                                          "id_region",
                                          "main_okved_group",
                                         ],
                            one_hot_max_size=256,
                            #l2_leaf_reg=3,
                            task_type="GPU",
                            model_size_reg=10,
                            random_seed=0,
                            has_time=True,
                         )
model.fit(X_train,y_train,
          eval_set=(X_test, y_test),
          plot=True,
          verbose=100,
          use_best_model=True,
          early_stopping_rounds=30,
         )
#model.save_model(f"model{model.learning_rate_}_{model.best_score_}.cbm")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.156962
0:	learn: 3.6851447	test: 3.1259070	best: 3.1259070 (0)	total: 83.4ms	remaining: 1m 23s
100:	learn: 2.4787158	test: 2.1981587	best: 2.1981587 (100)	total: 8.69s	remaining: 1m 17s
200:	learn: 2.4673754	test: 2.1866608	best: 2.1866608 (200)	total: 17.4s	remaining: 1m 9s
300:	learn: 2.4617855	test: 2.1816531	best: 2.1816531 (300)	total: 26.1s	remaining: 1m
400:	learn: 2.4581151	test: 2.1796347	best: 2.1796347 (400)	total: 34.8s	remaining: 52s
500:	learn: 2.4551717	test: 2.1767972	best: 2.1767818 (496)	total: 43.5s	remaining: 43.3s
600:	learn: 2.4528117	test: 2.1758499	best: 2.1758499 (600)	total: 52s	remaining: 34.5s
700:	learn: 2.4505431	test: 2.1745849	best: 2.1742891 (682)	total: 1m	remaining: 26s
bestTest = 2.174289054
bestIteration = 682
Shrink model to first 683 iterations.


In [40]:
model.get_feature_importance(prettified=True).head(60)

,Feature Id,Importances
0,dy1,21.991297
1,imax1,15.524298
2,imax2,11.040772
3,imean,9.060971
4,iq065,5.981192
5,imax4,5.765725
6,imax3,3.805906
7,imedian,3.471094
8,iq070,2.830560
9,iq080,2.600364


In [ ]:
model.get_feature_importance(prettified=True).tail(60)

In [41]:
%%time
innweeks0 = pl.read_parquet("data/target_series.parquet"
                            ).with_columns(                              
                                pl.col("week").cast(pl.Int8),
                                np.log1p(pl.col("target")),
                            )
innweeks0

innweeks0 = innweeks0.with_columns(
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(0).over("inn_id").alias("imax1"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(1).over("inn_id").alias("imax2"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(2).over("inn_id").alias("imax3"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(3).over("inn_id").alias("imax4"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(4).over("inn_id").alias("imax5"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(5).over("inn_id").alias("imax6"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(6).over("inn_id").alias("imax7"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(7).over("inn_id").alias("imax8"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(8).over("inn_id").alias("imax9"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(9).over("inn_id").alias("imax10"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(10).over("inn_id").alias("imax11"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(11).over("inn_id").alias("imax12"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(12).over("inn_id").alias("imax13"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(13).over("inn_id").alias("imax14"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(14).over("inn_id").alias("imax15"),
                      pl.col("target").rolling_max(window_size=1,min_periods=1).shift(15).over("inn_id").alias("imax16"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(17).over("inn_id").alias("imax17"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(18).over("inn_id").alias("imax18"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(19).over("inn_id").alias("imax19"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(20).over("inn_id").alias("imax20"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(21).over("inn_id").alias("imax21"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(22).over("inn_id").alias("imax22"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(23).over("inn_id").alias("imax23"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(24).over("inn_id").alias("imax24"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(25).over("inn_id").alias("imax25"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(26).over("inn_id").alias("imax26"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(27).over("inn_id").alias("imax27"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(28).over("inn_id").alias("imax28"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(29).over("inn_id").alias("imax29"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(30).over("inn_id").alias("imax30"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(31).over("inn_id").alias("imax31"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(32).over("inn_id").alias("imax32"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(33).over("inn_id").alias("imax33"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(34).over("inn_id").alias("imax34"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(35).over("inn_id").alias("imax35"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(36).over("inn_id").alias("imax36"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(37).over("inn_id").alias("imax37"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(38).over("inn_id").alias("imax38"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(39).over("inn_id").alias("imax39"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(40).over("inn_id").alias("imax40"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(41).over("inn_id").alias("imax41"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(42).over("inn_id").alias("imax42"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(43).over("inn_id").alias("imax43"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(44).over("inn_id").alias("imax44"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(45).over("inn_id").alias("imax45"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(46).over("inn_id").alias("imax46"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(47).over("inn_id").alias("imax47"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(48).over("inn_id").alias("imax48"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(49).over("inn_id").alias("imax49"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(50).over("inn_id").alias("imax50"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(51).over("inn_id").alias("imax51"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(52).over("inn_id").alias("imax52"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(53).over("inn_id").alias("imax53"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(54).over("inn_id").alias("imax54"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(55).over("inn_id").alias("imax55"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(56).over("inn_id").alias("imax56"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(57).over("inn_id").alias("imax57"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(58).over("inn_id").alias("imax58"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(59).over("inn_id").alias("imax59"),
                      
                      pl.col("target").rolling_max(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imax"),
                      pl.col("target").rolling_min(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imin"),
                      pl.col("target").rolling_mean(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imean"),
                      pl.col("target").rolling_median(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imedian"),
    
                      pl.col("target").rolling_quantile(quantile=0.95, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq095"),
                      pl.col("target").rolling_quantile(quantile=0.90, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq090"),
                      pl.col("target").rolling_quantile(quantile=0.85, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq085"),
                      pl.col("target").rolling_quantile(quantile=0.80, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq080"),
                      pl.col("target").rolling_quantile(quantile=0.75, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq075"),
                      pl.col("target").rolling_quantile(quantile=0.70, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq070"),
                      pl.col("target").rolling_quantile(quantile=0.65, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq065"),
                      pl.col("target").rolling_quantile(quantile=0.60, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq060"),
                      pl.col("target").rolling_quantile(quantile=0.55, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq055"),
    
                      pl.col("target").rolling_quantile(quantile=0.45, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq045"),
                      pl.col("target").rolling_quantile(quantile=0.40, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq040"),
                      pl.col("target").rolling_quantile(quantile=0.35, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq035"),
                      pl.col("target").rolling_quantile(quantile=0.30, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq030"),
                      pl.col("target").rolling_quantile(quantile=0.25, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq025"),
                      pl.col("target").rolling_quantile(quantile=0.20, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq020"),
                      pl.col("target").rolling_quantile(quantile=0.15, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq015"),
                      pl.col("target").rolling_quantile(quantile=0.10, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq010"),
                      pl.col("target").rolling_quantile(quantile=0.05, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq005"),
                     )
innweeks0 = innweeks0.group_by("inn_id").last().drop(["week","target"])
innweeks0

CPU times: user 16.1 s, sys: 5.96 s, total: 22.1 s
Wall time: 21 s


inn_id,imax1,imax2,imax3,imax4,imax5,imax6,imax7,imax8,imax9,imax10,imax11,imax12,imax13,imax14,imax15,imax16,imax,imin,imean,imedian,iq095,iq090,iq085,iq080,iq075,iq070,iq065,iq060,iq055,iq045,iq040,iq035,iq030,iq025,iq020,iq015,iq010,iq005
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""inn2583641""",11.886924,11.776353,12.277402,12.009699,11.781548,11.359734,12.439573,12.768457,11.617431,12.528783,12.994599,12.65533,12.144293,12.830565,14.100916,13.790074,14.100916,0.0,11.430436,12.366958,13.791227,13.566523,13.26157,13.104754,12.888677,12.88526,12.710744,12.531279,12.5099,12.144293,11.979227,11.798718,11.719746,11.617431,11.152916,10.968777,10.798522,0.0
"""inn1731164""",11.303852,11.692768,12.093958,10.575181,12.590662,11.915199,9.463919,11.403244,12.784963,11.308985,9.64584,11.847614,11.492428,12.469207,11.058369,12.5855,12.950805,0.0,11.465481,11.878194,12.784963,12.598376,12.5855,12.36448,12.339579,12.315217,12.096039,12.006957,11.946703,11.692768,11.571424,11.492428,11.316643,11.303852,11.050862,10.720394,10.31245,9.463919
"""inn2543328""",15.528783,14.200032,12.472983,13.084798,13.797001,13.239243,13.679216,10.226836,13.692455,14.81853,14.814425,11.832332,14.058024,12.987092,14.835861,12.641132,17.322908,0.0,14.843553,15.562885,16.957207,16.762126,16.717221,16.582363,16.500859,16.353415,16.158663,15.920852,15.791228,15.287796,14.81853,14.294654,14.058024,13.797001,13.539018,13.22158,12.987092,11.832332
"""inn226621""",12.847438,14.418693,10.822874,13.13739,11.329864,12.471434,0.0,13.111359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.127818,15.370809,0.0,10.533628,13.412226,15.232166,15.127818,14.787459,14.576899,14.469029,14.367787,14.20144,13.882603,13.782658,13.111359,12.935051,12.30151,11.887047,10.822874,0.0,0.0,0.0,0.0
"""inn4180311""",11.526072,13.629147,11.282955,6.902411,12.712955,9.514264,11.035253,11.263408,12.671747,12.804768,8.585022,12.317908,11.507147,11.615504,14.413349,14.073093,14.413349,6.902411,11.841873,12.096658,14.023445,13.51803,13.478254,13.209497,12.974309,12.804768,12.712955,12.594218,12.241989,11.625119,11.526072,11.350428,11.198952,10.977823,10.696564,9.880836,9.745537,9.512997
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn4777481""",13.666157,15.057422,15.424295,15.422807,12.826034,15.454721,15.200979,14.439313,14.490084,14.437643,15.158425,15.5162,14.848994,13.586493,15.201302,15.195924,15.812092,8.641822,14.712173,15.01167,15.683137,15.5162,15.454721,15.397018,15.256143,15.215365,15.195924,15.057422,15.017999,14.886158,14.813133,14.695615,14.633544,14.490084,14.359973,13.878713,13.666157,13.144604
"""inn3045238""",13.349638,11.983802,12.581854,11.33409,12.389866,12.519625,12.858148,11.505525,12.918763,12.979038,12.317235,12.333151,12.083897,12.007083,12.104698,12.655403,13.518068,0.0,11.875605,12.254104,13.411133,12.979038,12.890708,12.786803,12.666968,12.581854,12.415062,12.360602,12.317235,12.083897,12.046041,11.955482,11.898003,11.571337,11.489253,11.186735,10.973203,9.343961
"""inn3133789""",12.653015,12.23142,11.63473,11.04886,13.027358,11.528621,11.913224,13.077054,11.584402,12.476407,12.695221,11.278221,8.83213,9.3497,12.434971,11.620486,13.93748,8.83213,11.790833,11.913224,13.54802,13.027358,12.846918,12.695221,12.57187,12.504535,12.443089,12.295757,12.28167,11.63473,11.584402,11.480911,11.346043,11.278221,11.133714,10.216942,9.888532,9.432333


In [42]:
X0=pl.read_csv("data/sample_submit.csv"
           ).with_columns( pl.col("week").cast(pl.Int8)
            ).join(weeks,
                   on="week",
                   how="left"
                  ).join(profiles,on="inn_id",
                         how="left"
                        ).join(innweeks0,on="inn_id",
                         how="left"
                        ).select(X_train.columns).to_pandas()
X0

,inn_id,imax,imin,imean,imedian,iq095,iq090,iq085,iq080,iq075,...,year2,month2,day2,dy2,m,y,ipul,id_region,main_okved_group,diff_datopen_report_date_flg
0,inn1000051,15.821142,0.000000,10.398649,11.644807,14.972081,14.298071,13.952158,13.529218,13.151568,...,2024,8,11,223,0,0,0,45,68,96.0
1,inn1000051,15.821142,0.000000,10.398649,11.644807,14.972081,14.298071,13.952158,13.529218,13.151568,...,2024,8,18,230,0,0,0,45,68,96.0
2,inn1000051,15.821142,0.000000,10.398649,11.644807,14.972081,14.298071,13.952158,13.529218,13.151568,...,2024,8,25,237,0,0,0,45,68,96.0
3,inn1000051,15.821142,0.000000,10.398649,11.644807,14.972081,14.298071,13.952158,13.529218,13.151568,...,2024,9,1,244,1,0,0,45,68,96.0
4,inn1000051,15.821142,0.000000,10.398649,11.644807,14.972081,14.298071,13.952158,13.529218,13.151568,...,2024,9,8,251,0,0,0,45,68,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623551,inn999886,17.231515,11.500407,16.130318,16.199541,17.025046,16.853691,16.799984,16.786354,16.653305,...,2024,9,29,272,0,0,1,22,45,192.0
623552,inn999886,17.231515,11.500407,16.130318,16.199541,17.025046,16.853691,16.799984,16.786354,16.653305,...,2024,10,6,279,1,0,1,22,45,192.0
623553,inn999886,17.231515,11.500407,16.130318,16.199541,17.025046,16.853691,16.799984,16.786354,16.653305,...,2024,10,13,286,0,0,1,22,45,192.0
623554,inn999886,17.231515,11.500407,16.130318,16.199541,17.025046,16.853691,16.799984,16.786354,16.653305,...,2024,10,20,293,0,0,1,22,45,192.0


In [ ]:
#model.load_model("model0.15696200728416443_{'learn': {'RMSE': 2.1840243465326776}, 'validation': {'RMSE': 2.4295050367945095}}.cbm")
model

In [43]:
np.expm1( np.clip(model.predict(X0), 0, np.inf) )

array([  517808.77825236,   491555.69443525,   416215.62789398, ...,
       14991280.32870075, 16328959.53002461, 16458508.79122541])

In [44]:
Xp=pl.read_csv("data/sample_submit.csv"
           ).to_pandas()
Xp["predict"]=np.expm1( np.clip(model.predict(X0), 0, np.inf) )
Xp.to_csv("pred2_02a.csv")
Xp

,inn_id,week,predict
0,inn1000051,106,5.178088e+05
1,inn1000051,107,4.915557e+05
2,inn1000051,108,4.162156e+05
3,inn1000051,109,3.864073e+05
4,inn1000051,110,5.134595e+05
...,...,...,...
623551,inn999886,113,1.659222e+07
623552,inn999886,114,1.015723e+07
623553,inn999886,115,1.499128e+07
623554,inn999886,116,1.632896e+07
